In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [3]:
#1754884 record,1053282 with coupon_id,9738 coupon. date_received:20160101~20160615,date:20160101~20160630, 539438 users, 8415 merchants
off_train = pd.read_csv('data/ccf_offline_stage1_train.csv',header=None)
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
#2050 coupon_id. date_received:20160701~20160731, 76309 users(76307 in trainset, 35965 in online_trainset), 1559 merchants(1558 in trainset)
off_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv',header=None)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']
#11429826 record(872357 with coupon_id),762858 user(267448 in off_train)
on_train = pd.read_csv('data/ccf_online_stage1_train.csv',header=None)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']

c:\users\xpc\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\xpc\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
dataset3 = off_test
feature3 = off_train[((off_train.date>='20160315')&(off_train.date<='20160630'))|((off_train.date=='null')&(off_train.date_received>='20160315')&(off_train.date_received<='20160630'))]
dataset2 = off_train[(off_train.date_received>='20160515')&(off_train.date_received<='20160615')]
feature2 = off_train[(off_train.date>='20160201')&(off_train.date<='20160514')|((off_train.date=='null')&(off_train.date_received>='20160201')&(off_train.date_received<='20160514'))]
dataset1 = off_train[(off_train.date_received>='20160414')&(off_train.date_received<='20160514')]
feature1 = off_train[(off_train.date>='20160101')&(off_train.date<='20160413')|((off_train.date=='null')&(off_train.date_received>='20160101')&(off_train.date_received<='20160413'))]

In [15]:
t = dataset3[['user_id']]
t['this_month_user_receive_all_coupon_count'] = 1
t = t.groupby('user_id').agg('sum').reset_index()
print(t[:5])

   user_id  this_month_user_receive_all_coupon_count
0  1000020                                         1
1  1000026                                         1
2  1000452                                         1
3  1000510                                         1
4   100057                                         1


c:\users\xpc\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
t1 = dataset3[['user_id','coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
#按照user_id和coupon_id进行分组
#统计每个用户，使用不同优惠券的次数
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()
print(t1[:5])

   user_id coupon_id  this_month_user_receive_same_coupon_count
0  1000020     13602                                          1
1  1000026     13602                                          1
2  1000452      9983                                          1
3  1000510     10418                                          1
4   100057      2601                                          1


c:\users\xpc\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
t2 = dataset3[['user_id','coupon_id','date_received']]
t2.date_received = t2.date_received.astype('str')
# 按照user_id','coupon_id排序后，提出来date_received，进行agg运算
# agg运算：用冒号连接起来
t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
print(t2)

c:\users\xpc\appdata\local\programs\python\python36\lib\site-packages\pandas\core\generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


        user_id  coupon_id      date_received
0       1000020      13602           20160731
1       1000026      13602           20160729
2       1000452       9983           20160727
3       1000510      10418           20160701
4        100057       2601           20160708
5       1000651      13602           20160728
6       1000884      10438           20160714
7       1000907       1904           20160703
8       1000936       4203           20160710
9       1000986      12429           20160701
10      1001023      13602           20160723
11      1001176      13181           20160731
12      1001176        361           20160709
13      1001176       3992           20160731
14       100122      12735           20160714
15       100122      13602           20160727
16      1001240      10418           20160706
17      1001240      13602           20160711
18      1001240       2978           20160706
19      1001257      11799           20160711
20      1001302      13602        

In [30]:
#apply会返回每个优惠券的使用次数
t2['receive_number'] = t2.date_received.apply(lambda s:len(s.split(':')))
#筛出使用次数大于1次的数据
t2 = t2[t2.receive_number>1]
#对max_date_received赋值为最近一次的使用时间
t2['max_date_received'] = t2.date_received.apply(lambda s:max([int(d) for d in s.split(':')]))
#对min_date_received赋值为最早一次的使用时间
t2['min_date_received'] = t2.date_received.apply(lambda s:min([int(d) for d in s.split(':')]))
# 重新定义t2为以下项目
t2 = t2[['user_id','coupon_id','max_date_received','min_date_received']]

In [33]:
t3 = dataset3[['user_id','coupon_id','date_received']]
#merge，将两个数据集合并
#将t2和t3在['user_id','coupon_id']上进行左帧合并，即根据t3合并t2的user_id','coupon_id
#t2[['user_id','coupon_id','max_date_received','min_date_received']]
#t3[['user_id','coupon_id','date_received']]
#因此合并方式为：找到每个用户每张优惠券的消费时间和对应券的max_date_received与min_date_received
t3 = pd.merge(t3,t2,on=['user_id','coupon_id'],how='left')
#t3的this_month_user_receive_same_coupon_lastone项目设置为:此用户消费本张优惠券与最近一次消费本张优惠券的间隔
t3 = t3.apply(pd.to_numeric, args=('coerce',))
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received
#此用户消费本张优惠券与第一次消费本张优惠券的间隔
t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received - t3.min_date_received
def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once
    
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
t3 = t3[['user_id','coupon_id','date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]

In [34]:
print(t3)

          user_id  coupon_id  date_received  \
0             NaN        NaN            NaN   
1       4129537.0     9983.0     20160712.0   
2       6949378.0     3429.0     20160706.0   
3       2166529.0     6928.0     20160727.0   
4       2166529.0     1808.0     20160727.0   
5       6172162.0     6500.0     20160708.0   
6       4005121.0     9983.0     20160706.0   
7       4347394.0     9983.0     20160716.0   
8       3094273.0    13602.0     20160727.0   
9       5139970.0     9983.0     20160729.0   
10      3237121.0    13602.0     20160703.0   
11      6224386.0     9983.0     20160716.0   
12      6488578.0    13602.0     20160712.0   
13      4164865.0     9983.0     20160703.0   
14      4164865.0     8059.0     20160706.0   
15      5468674.0     9983.0     20160713.0   
16      6258178.0     9144.0     20160706.0   
17      3659521.0     7341.0     20160727.0   
18      3659521.0    13181.0     20160717.0   
19      3659521.0    13602.0     20160718.0   
20      73333